# SSNE Miniproject 3
### 318703 Tomasz Owienko
### 318718 Anna Schäfer
### Grupa piątek

In [7]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import pytorch_lightning as pl
import torch.nn.functional as F
import numpy as np

In [8]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_float32_matmul_precision('medium')

In [11]:
transform = transforms.Compose(
    [
        # transforms.RandomHorizontalFlip(0.5),
        # transforms.RandomRotation(24),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

train_image_folder = torchvision.datasets.ImageFolder('train', transform=transform)

n = len(train_image_folder)
print(n)
evaluation_range = int(n / 10)
index_list = np.arange(n)
np.random.shuffle(index_list)

train_set = torch.utils.data.Subset(train_image_folder, index_list[evaluation_range:])
evaluation_set = torch.utils.data.Subset(train_image_folder, index_list[:evaluation_range])

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=512, shuffle=True, num_workers=8, pin_memory=True
)
evaluation_loader = torch.utils.data.DataLoader(
    evaluation_set, batch_size=16, shuffle=False, num_workers=8, pin_memory=True
)

classes = train_image_folder.find_classes('train')[0]
print(classes, len(classes))

88011
['acoustic', 'antenna', 'bacteria', 'battery', 'bean', 'beetle', 'bicycle', 'birch', 'bird', 'bomb', 'bread', 'bridge', 'camera', 'carbon', 'cat', 'corn', 'crab', 'crocodilian', 'echinoderm', 'egg', 'elephant', 'fish', 'flower', 'frog', 'fungus', 'gauge', 'hammer', 'icecream', 'kangaroo', 'memorial', 'monkey', 'motor', 'nest', 'palm', 'pizza', 'pot', 'printer', 'saw', 'snake', 'spice', 'spider', 'spoon', 'squash', 'swine', 'tea', 'tomato', 'towel', 'truck', 'turtle', 'worm'] 50


In [12]:
class ImageClassifier(pl.LightningModule):
    def __init__(self, num_classes=50):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        self.log('test_loss', loss)
        return loss

In [17]:
model = ImageClassifier()
trainer = pl.Trainer(max_epochs=100, accelerator="gpu", devices='auto')
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=evaluation_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type      | Params
--------------------------------------
0 | conv1   | Conv2d    | 896   
1 | conv2   | Conv2d    | 18.5 K
2 | conv3   | Conv2d    | 73.9 K
3 | conv4   | Conv2d    | 295 K 
4 | pool    | MaxPool2d | 0     
5 | fc1     | Linear    | 4.2 M 
6 | fc2     | Linear    | 524 K 
7 | fc3     | Linear    | 25.7 K
8 | dropout | Dropout   | 0     
--------------------------------------
5.1 M     Trainable params
0         Non-trainable params
5.1 M     Total params
20.537    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [19]:
%load_ext tensorboard
%tensorboard --logdir=lightning_logs